In [1]:
# Setup
import jinja2
import numpy as np
import os

OutDir = os.path.join(os.path.pardir,os.path.pardir,'data','results')


def FormatNumber(x,precision):
    if type(x) == np.ndarray:
        return np.array([FormatNumber(xi) for xi in x])
    else:       
        if x >= 0:
            formatstring = '{x:.' + str(precision) + 'f}'
            return formatstring.format(x = x)
        else:
            return '--'


env = jinja2.Environment(
        "%<", ">%",
        "<<", ">>",
        "[§", "§]",
        loader=jinja2.FileSystemLoader(".")
        )


RowTemplate =   '(<<rownum>>) & <<rowname>>  & <<bwo>> & <<bw>> &  & <<tauwo>> & <<tauw>> & & <<rrwo>> & <<rrw>> \\\\'

template = env.from_string(RowTemplate)

def RepRate(b,tau):
    return 2 * b **(1.0/(1-tau)) -1

def GetRow(Version, rownum,rowname):
    
    with np.load(os.path.join(OutDir,'SearchResults_v'+str(Version)+'.npz')) as X:
        OptCyc = X['OptCyc']
        OptSS = X['OptSS']
    
    return template.render(rownum = rownum,
                    rowname = rowname,
                    bwo = FormatNumber(OptSS[0],3),
                    bw  = FormatNumber(OptCyc[0],3),
                    tauwo = FormatNumber(OptSS[1],3),
                    tauw  = FormatNumber(OptCyc[1],3),
                    rrwo = FormatNumber(RepRate(OptSS[0],OptSS[1]),2),
                    rrw  = FormatNumber(RepRate(OptCyc[0],OptCyc[1]),2)
                          )
                



In [2]:
fname = os.path.join(OutDir,'RobustnessTable.tex')
f = open(fname, "w")

f.write(GetRow(1,'i','Baseline'))
f.write(os.linesep)
f.write(GetRow(8,'ii','Flexible prices'))
f.write(os.linesep)
f.write(GetRow(3,'iii','Aggressive monetary policy'))
f.write(os.linesep)
f.write(GetRow(10,'iv','Acyclical purchases'))
f.write(os.linesep)
f.write(GetRow(12,'v','No $Q^u$'))
f.write(os.linesep)
f.write(GetRow(16,'vi','No skill risk'))                     
f.write(os.linesep)


                     
f.close()